In [1]:
import tensorflow as tf
import tangent
tf.enable_eager_execution()

/home/act65/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [82]:
n = 8
x = tf.random_normal([10, n])
W = tf.random_normal([n,2])
W

<tf.Tensor: id=1734, shape=(8, 2), dtype=float32, numpy=
array([[ 1.30135   , -0.63694954],
       [ 1.1950794 , -1.4337986 ],
       [ 1.3677213 ,  1.8533424 ],
       [ 0.22865246,  0.97131443],
       [-1.3661206 , -0.93618524],
       [-0.13868548, -0.20348763],
       [-0.23222283, -0.68603843],
       [-0.12158792, -0.9232079 ]], dtype=float32)>

In [87]:
def matmul(x, y):
    x = tf.expand_dims(x, -1)
    y = tf.expand_dims(y, 0)
    return tf.reduce_sum(tf.multiply(x, y), axis=1)
matmul(x, W).shape

TensorShape([Dimension(10), Dimension(2)])

In [96]:
# ForwardNotImplementedError: Forward mode for function "gradients" is not yet implemented.
# ForwardNotImplementedError: Forward mode for function "matmul" is not yet implemented.

In [95]:
def energy_fn(x, W, b):
    h = tf.nn.sigmoid(x)

    energy = -0.5*tf.reduce_sum(tf.matmul(h, tf.matmul(W, h, transpose_b=True)), axis=1)
    energy += 0.5*tf.reduce_sum(tf.square(h*b), axis=1)
    energy += 0.5*tf.reduce_sum(x**2, axis=1)

    return tf.reduce_mean(energy)

def step(x, W):
    return matmul(x, W)

forward_dfdx = tangent.autodiff(step, mode='forward')

In [89]:
# forward_dfdx(x, W, tf.ones_like(x))

In [90]:
def forward(state, n_steps=1):
    
    def _step(i, state, dydx):
        # a wrapper for self.step(...)
        new_state = step(state, W)
        return i + 1, new_state, forward_dfdx(x, W, dx=dydx)

    with tf.name_scope('forward'):
        while_condition = lambda i, j, k : tf.less(i, n_steps)
        i = tf.constant(0)
        dydx = tf.ones_like(state)
        new_i_, new_state, new_dydx = tf.while_loop(
            while_condition, 
            _step, 
            loop_vars=[i, state, dydx], 
            back_prop=False)

        return new_state, new_dydx

In [91]:
y, dy = forward(x)

In [92]:
dy.shape

TensorShape([Dimension(10), Dimension(2)])